In [1]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
import selenium.webdriver
import os # too access user/pwd in .bash_profile
from loseit_parser import LoseItPostParser
from scraper_1 import sign_in

# get our sample mongo documents
mc = MongoClient()
db = mc['lose_it']
db.collection_names()
coll = db['shard_long_topic']
landing, subpage = coll.find({}, limit=2)

# grab url and html inputs from sample documents
ldg_url = landing['url']
ldg_html = landing['html']
sub_url = subpage['url']
sub_html = subpage['html']
last_pg, = coll.find({
    'url': 'https://forums.loseit.com/posts/list/1965/18808.page'
    })
last_url = last_pg['url']
last_html = last_pg['html']
"""TEACHING MOMENT: line 13 -- NEEDED COMMA to tell python to unpack
the results of the pymongo cursor."""

In [28]:
parse_landing = LoseItPostParser(ldg_html, ldg_url)
parse_sub = LoseItPostParser(sub_html, sub_url)
parse_last = LoseItPostParser(last_html, last_url)

parse_landing._nav_to_header_parent()

parse_landing._nav_to_posts_parent()

parse_landing.parse_library()
parse_landing.parse_last_pg_url()

In [29]:
parse_landing.compile_records()

landing_bin = parse_landing.posts_lst
landing_bin[12]

{'author_id': 1779015,
 'date_posted': u'01/22/2012',
 'joined': u'01/11/2012',
 'library': u'Teams, Contests, and Fun',
 'name': u'Eleslie Patterson',
 'page_idx': 1,
 'post_body': u'lol dont be a baby... man up...',
 'post_idx': 13,
 'profile_url': u'http://www.loseit.com/profile?id=1779015&name=Eleslie+Patterson',
 'topic': u'Need motivators!!! How about some friendly WEEKLY challenges?',
 'topic_idx': 18808,
 'ttl_posts': 395}

In [5]:
coll2 = db['shard_cherrypicked']

In [6]:
grumpy, = coll2.find({
    'url': 'https://forums.loseit.com/posts/list/6454.page'
    })
grumpy_url = grumpy['url']
grumpy_html = grumpy['html']

In [451]:
parse_grumpy = LoseItPostParser(grumpy_html, grumpy_url)

In [ ]:
parse_grumpy.parse_page()

parse_grumpy.compile_records()

In [460]:
parse_grumpy.posts_lst[14]

{'author_id': 745201,
 'date_posted': u'01/12/2011',
 'joined': u'01/02/2011',
 'library': u'The Kitchen Sink',
 'name': u'Becky',
 'page_idx': 1,
 'post_body': "There's a citation!",
 'post_idx': 15,
 'profile_url': u'http://www.loseit.com/profile?id=745201&name=Becky',
 'topic': u'The Grumpy McRant-Rant Thread',
 'topic_idx': 6454,
 'ttl_posts': 32}

In [452]:
basecamp = parse_grumpy.posts_parent

In [106]:
print(basecamp)

<tr>
<th class="thcornerl" nowrap="nowrap"></th>
<th class="thtop" height="26" nowrap="nowrap" width="150">Author</th>
<th class="thtop" nowrap="nowrap" width="100%">Message</th>
<th class="thcornerr" nowrap="nowrap"></th>
</tr>


In [107]:
text = (basecamp.find_next_sibling('tr')
         .find_next_sibling('tr')
        .find('a')['href']
#         .find('span', 'post_text_60155').text.strip()
       )

text

u'http://www.loseit.com/profile?id=494072&name=Jon+S'

In [403]:
platform = basecamp
i = 0

In [446]:
def iterate_trs(bs4_obj, i):
    bs4_obj = bs4_obj.find_next_sibling('tr')
    i += 1
    return i, bs4_obj

i, platform = iterate_trs(platform, i)
print i
print platform

42
<tr>
<!-- Username -->
<td class="row1 borderleft"> </td>
<td align="left" class="row1" rowspan="2" valign="top">
<div class="genmed" style="margin-bottom:5px;">
<a href="http://www.loseit.com/profile?id=494072&amp;name=Jon+S" style="color:#0064C7;cursor:pointer;font-size:9pt;font-weight:bold;">Jon S</a>
</div>
<span class="gensmall">
<img alt="[Avatar]" border="0" src="https://d21zgfprgikg74.cloudfront.net/connect/profilepics/24530B61C81840B49179A04AAADE82C0/small.png"/>
<div>Joined: 07/11/2010</div>
<div>
	Messages: 2886
        </div>
<div class="offline">Offline </div>
</span> </td>
<!-- Message -->
<td class="row1" id="post_text_60281" valign="top">
<span class="postbody">
<blockquote>
<div>
<cite>Debbie Lee wrote:</cite>I'm grumpy too!  I recently had to move back in with my parents.  That's reason enough to be grumpy - but it gets worse.  Everyone here at home is overweight, yet I'm the only one who seems to CARE about this!  I'm trying to eat right and exercise, but I have z

In [100]:
stats = (basecamp.find_next_sibling('tr')
         .find_next_sibling('tr')
         .find_next_siblings('tr')
         #.find('span', 'postbody')
         #.find_next_sibling('span')
         #.find('div')
        )

stats[0]

<tr>\n<td class="row1 borderleft">\xa0</td>\n<td class="row1" height="28" nowrap="nowrap" valign="bottom" width="100%">\n<table border="0" cellpadding="0" cellspacing="0" width="100%">\n<tbody><tr>\n<td nowrap="nowrap" valign="middle">\n</td>\n<td align="right">\n<script type="text/javascript">karmaPointsCombo(60155)</script>\n</td>\n</tr>\n</tbody></table> </td>\n<td class="row1 borderright">\xa0</td>\n</tr>

In [448]:
# This gets us the author cited
cited = platform.find('cite').text.strip().split(' ')[:-1]
(' ').join(cited)

u'Debbie Lee'

In [447]:
if platform.find('cite'): print "Yep, there's a citation!"
else: print "Nope, no citation."

Yep, there's a citation!


In [351]:
# This gets us the citation
citation = (platform.find('td')
            .find_next_sibling('td')
            .find_next_sibling('td')
            .find('div')
           ).text.strip()

print citation[(citation.index(':') + 1):] # cuts off cited named

I'm grumpy too!  I recently had to move back in with my parents.  That's reason enough to be grumpy - but it gets worse.  Everyone here at home is overweight, yet I'm the only one who seems to CARE about this!  I'm trying to eat right and exercise, but I have zero support from my family.  They buy cookies and cheesecake and leave it in the kitchen for me to stare at and drool.  They are making this harder than it needs to be!!!


In [352]:
# This gets the body
citn_body = (platform.find('td')
            .find_next_sibling('td')
            .find_next_sibling('td')
            .find('span')
           ).text.strip()

body = citn_body[citn_body.index('\n\n\n'):].strip()
body

u'OMG, I totally feel you!!  I am irked at my parents right now for the same thing. My parents have been overweight their entire lives, but have no problem telling someone how fat they have gotten.  They lose five stinkin\' pounds and they act like they are ready to walk the runway.  Then, ok, you listen to them and try to eat better, but you go over to their house for a family event and when you try to turn something down because you really shouldn\'t eat it, they reposnd with irritating little quips like, "Oh you can\'t eat anything on your diet" or "Oh you\'re no fun anymore." ARRRRRRGGGGGHHHHHH!!! And they do it to everyone, including my Aunt who was recently diagnosed as diabetic. When she is bad with her diet, my mother criticizes, but when she is good with her diet my mother calls her a stick in the mud.  EVERYTHING leaves may parents\' house fat... even the pets!!\n\n Ok, I feel better now.'

In [72]:
parse_grumpy.compile_records()

In [73]:
parse_grumpy.posts_lst[9]['post_body']

u"I'm grumpy too!  I recently had to move back in with my parents.  That's reason enough to be grumpy - but it gets worse.  Everyone here at home is overweight, yet I'm the only one who seems to CARE about this!  I'm trying to eat right and exercise, but I have zero support from my family.  They buy cookies and cheesecake and leave it in the kitchen for me to stare at and drool.  They are making this harder than it needs to be!!!"

In [ ]:
"""PSEUDO CODE for writing topics to relevant dbs.
"""
for document in kitchen_mongo: